##### Copyright 2020 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");

In [1]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Meta-Dataset leaderboard

This notebook computes leaderboard tables for different models on Meta-Dataset.

Results for each model in each training setting (ImageNet-only or all datasets) are defined in a different DataFrame. This script aggregates the data in one DataFrame, ranks the models in each setting (using a statistical test for equality), and produces the final tables.

In [2]:
import re
import textwrap

import numpy as np
import pandas as pd
from IPython import display

In [3]:
# Explicit list of evaluation datasets.
# ILSVRC (valid) is included for completeness, but does not have to be reported.
datasets = [
    "ILSVRC (valid)",
    "ILSVRC (test)",
    "Omniglot",
    "Aircraft",
    "Birds",
    "Textures",
    "QuickDraw",
    "Fungi",
    "VGG Flower",
    "Traffic signs",
    "MSCOCO"
]

# Explicit list of articles and references, filled in throughout the notebook.
references = []

## Results from Triantafillou et al. (2020)

In [4]:
ref = ("Triantafillou et al. (2020)",
       "Eleni Triantafillou, Tyler Zhu, Vincent Dumoulin, Pascal Lamblin, "
       "Utku Evci, Kelvin Xu, Ross Goroshin, Carles Gelada, Kevin Swersky, "
       "Pierre-Antoine Manzagol, Hugo Larochelle; "
       "[_Meta-Dataset: A Dataset of Datasets for Learning to Learn from Few "
       "Examples_](https://arxiv.org/abs/1903.03096); ICLR 2020.")
references.append(ref)
# display.display(display.Markdown(ref[1]))

### k-NN (`baseline`)

In [5]:
baseline_imagenet_df = pd.DataFrame(
    columns=['mean (%)', '95% CI', '# episodes'],
    index=datasets
)
baseline_imagenet_df['# episodes'] = 600

In [6]:
baseline_imagenet_df.loc[datasets[1:], ['mean (%)', '95% CI']] = [
    [41.03, 1.01],
    [37.07, 1.15],
    [46.81, 0.89],
    [50.13, 1.00],
    [66.36, 0.75],
    [32.06, 1.08],
    [36.16, 1.02],
    [83.10, 0.68],
    [44.59, 1.19],
    [30.38, 0.99]
]

In [7]:
baseline_imagenet_df

,mean (%),95% CI,# episodes
ILSVRC (valid),NaN,NaN,600
ILSVRC (test),41.03,1.01,600
Omniglot,37.07,1.15,600
Aircraft,46.81,0.89,600
Birds,50.13,1,600
Textures,66.36,0.75,600
QuickDraw,32.06,1.08,600
Fungi,36.16,1.02,600
VGG Flower,83.1,0.68,600
Traffic signs,44.59,1.19,600


In [8]:
baseline_all_df = pd.DataFrame(
    columns=['mean (%)', '95% CI', '# episodes'],
    index=datasets
)
baseline_all_df['# episodes'] = 600
baseline_all_df.loc[datasets[1:], ['mean (%)', '95% CI']] = [
    [38.55, 0.94],
    [74.60, 1.08],
    [64.98, 0.82],
    [66.35, 0.92],
    [63.58, 0.79],
    [44.88, 1.05],
    [37.12, 1.06],
    [83.47, 0.61],
    [40.11, 1.10],
    [29.55, 0.96]
]
baseline_all_df

,mean (%),95% CI,# episodes
ILSVRC (valid),NaN,NaN,600
ILSVRC (test),38.55,0.94,600
Omniglot,74.6,1.08,600
Aircraft,64.98,0.82,600
Birds,66.35,0.92,600
Textures,63.58,0.79,600
QuickDraw,44.88,1.05,600
Fungi,37.12,1.06,600
VGG Flower,83.47,0.61,600
Traffic signs,40.11,1.1,600


### Finetune (`baselinefinetune`)

In [9]:
baselineft_imagenet_df = pd.DataFrame(
    columns=['mean (%)', '95% CI', '# episodes'],
    index=datasets
)
baselineft_imagenet_df['# episodes'] = 600
baselineft_imagenet_df.loc[datasets[1:], ['mean (%)', '95% CI']] = [
    [45.78, 1.10],
    [60.85, 1.58],
    [68.69, 1.26],
    [57.31, 1.26],
    [69.05, 0.90],
    [42.60, 1.17],
    [38.20, 1.02],
    [85.51, 0.68],
    [66.79, 1.31],
    [34.86, 0.97]
]
baselineft_imagenet_df

,mean (%),95% CI,# episodes
ILSVRC (valid),NaN,NaN,600
ILSVRC (test),45.78,1.1,600
Omniglot,60.85,1.58,600
Aircraft,68.69,1.26,600
Birds,57.31,1.26,600
Textures,69.05,0.9,600
QuickDraw,42.6,1.17,600
Fungi,38.2,1.02,600
VGG Flower,85.51,0.68,600
Traffic signs,66.79,1.31,600


In [10]:
baselineft_all_df = pd.DataFrame(
    columns=['mean (%)', '95% CI', '# episodes'],
    index=datasets
)
baselineft_all_df['# episodes'] = 600
baselineft_all_df.loc[datasets[1:], ['mean (%)', '95% CI']] = [
    [43.08, 1.08],
    [71.11, 1.37],
    [72.03, 1.07],
    [59.82, 1.15	],
    [69.14, 0.85],
    [47.05, 1.16	],
    [38.16, 1.04],
    [85.28, 0.69],
    [66.74, 1.23],
    [35.17, 1.08]
]
baselineft_all_df

,mean (%),95% CI,# episodes
ILSVRC (valid),NaN,NaN,600
ILSVRC (test),43.08,1.08,600
Omniglot,71.11,1.37,600
Aircraft,72.03,1.07,600
Birds,59.82,1.15,600
Textures,69.14,0.85,600
QuickDraw,47.05,1.16,600
Fungi,38.16,1.04,600
VGG Flower,85.28,0.69,600
Traffic signs,66.74,1.23,600


### MatchingNet (`matching`)

In [11]:
matching_imagenet_df = pd.DataFrame(
    columns=['mean (%)', '95% CI', '# episodes'],
    index=datasets
)
matching_imagenet_df['# episodes'] = 600
matching_imagenet_df.loc[datasets[1:], ['mean (%)', '95% CI']] = [
    [45.00, 1.10],
    [52.27, 1.28],
    [48.97, 0.93],
    [62.21, 0.95],
    [64.15, 0.85],
    [42.87, 1.09],
    [33.97, 1.00],
    [80.13, 0.71],
    [47.80, 1.14],
    [34.99, 1.00]
]
matching_imagenet_df

,mean (%),95% CI,# episodes
ILSVRC (valid),NaN,NaN,600
ILSVRC (test),45,1.1,600
Omniglot,52.27,1.28,600
Aircraft,48.97,0.93,600
Birds,62.21,0.95,600
Textures,64.15,0.85,600
QuickDraw,42.87,1.09,600
Fungi,33.97,1,600
VGG Flower,80.13,0.71,600
Traffic signs,47.8,1.14,600


In [12]:
matching_all_df = pd.DataFrame(
    columns=['mean (%)', '95% CI', '# episodes'],
    index=datasets
)
matching_all_df['# episodes'] = 600
matching_all_df.loc[datasets[1:], ['mean (%)', '95% CI']] = [
    [36.08, 1.00],
    [78.25, 1.01],
    [69.17, 0.96],
    [56.40, 1.00],
    [61.80, 0.74],
    [60.81, 1.03],
    [33.70, 1.04],
    [81.90, 0.72],
    [55.57, 1.08],
    [28.79, 0.96]
]
matching_all_df

,mean (%),95% CI,# episodes
ILSVRC (valid),NaN,NaN,600
ILSVRC (test),36.08,1,600
Omniglot,78.25,1.01,600
Aircraft,69.17,0.96,600
Birds,56.4,1,600
Textures,61.8,0.74,600
QuickDraw,60.81,1.03,600
Fungi,33.7,1.04,600
VGG Flower,81.9,0.72,600
Traffic signs,55.57,1.08,600


### ProtoNet (`prototypical`)

In [13]:
prototypical_imagenet_df = pd.DataFrame(
    columns=['mean (%)', '95% CI', '# episodes'],
    index=datasets
)
prototypical_imagenet_df['# episodes'] = 600
prototypical_imagenet_df.loc[datasets[1:], ['mean (%)', '95% CI']] = [
    [50.50, 1.08],
    [59.98, 1.35],
    [53.10, 1.00],
    [68.79, 1.01],
    [66.56, 0.83],
    [48.96, 1.08],
    [39.71, 1.11],
    [85.27, 0.77],
    [47.12, 1.10],
    [41.00, 1.10]
]
prototypical_imagenet_df

,mean (%),95% CI,# episodes
ILSVRC (valid),NaN,NaN,600
ILSVRC (test),50.5,1.08,600
Omniglot,59.98,1.35,600
Aircraft,53.1,1,600
Birds,68.79,1.01,600
Textures,66.56,0.83,600
QuickDraw,48.96,1.08,600
Fungi,39.71,1.11,600
VGG Flower,85.27,0.77,600
Traffic signs,47.12,1.1,600


In [14]:
prototypical_all_df = pd.DataFrame(
    columns=['mean (%)', '95% CI', '# episodes'],
    index=datasets
)
prototypical_all_df['# episodes'] = 600
prototypical_all_df.loc[datasets[1:], ['mean (%)', '95% CI']] = [
    [44.50, 1.05],
    [79.56, 1.12],
    [71.14, 0.86],
    [67.01, 1.02],
    [65.18, 0.84],
    [64.88, 0.89],
    [40.26, 1.13],
    [86.85, 0.71],
    [46.48, 1.00],
    [39.87, 1.06]
]
prototypical_all_df

,mean (%),95% CI,# episodes
ILSVRC (valid),NaN,NaN,600
ILSVRC (test),44.5,1.05,600
Omniglot,79.56,1.12,600
Aircraft,71.14,0.86,600
Birds,67.01,1.02,600
Textures,65.18,0.84,600
QuickDraw,64.88,0.89,600
Fungi,40.26,1.13,600
VGG Flower,86.85,0.71,600
Traffic signs,46.48,1,600


### fo-MAML (`maml`)

In [15]:
maml_imagenet_df = pd.DataFrame(
    columns=['mean (%)', '95% CI', '# episodes'],
    index=datasets
)
maml_imagenet_df['# episodes'] = 600
maml_imagenet_df.loc[datasets[1:], ['mean (%)', '95% CI']] = [
    [45.51, 1.11],
    [55.55, 1.54],
    [56.24, 1.11],
    [63.61, 1.06],
    [68.04, 0.81],
    [43.96, 1.29],
    [32.10, 1.10],
    [81.74, 0.83],
    [50.93, 1.51],
    [35.30, 1.23]
]
maml_imagenet_df

,mean (%),95% CI,# episodes
ILSVRC (valid),NaN,NaN,600
ILSVRC (test),45.51,1.11,600
Omniglot,55.55,1.54,600
Aircraft,56.24,1.11,600
Birds,63.61,1.06,600
Textures,68.04,0.81,600
QuickDraw,43.96,1.29,600
Fungi,32.1,1.1,600
VGG Flower,81.74,0.83,600
Traffic signs,50.93,1.51,600


In [16]:
maml_all_df = pd.DataFrame(
    columns=['mean (%)', '95% CI', '# episodes'],
    index=datasets
)
maml_all_df['# episodes'] = 600
maml_all_df.loc[datasets[1:], ['mean (%)', '95% CI']] = [
    [37.83, 1.01],
    [83.92, 0.95],
    [76.41, 0.69],
    [62.43, 1.08],
    [64.16, 0.83],
    [59.73, 1.10],
    [33.54, 1.11],
    [79.94, 0.84],
    [42.91, 1.31],
    [29.37, 1.08]
]
maml_all_df

,mean (%),95% CI,# episodes
ILSVRC (valid),NaN,NaN,600
ILSVRC (test),37.83,1.01,600
Omniglot,83.92,0.95,600
Aircraft,76.41,0.69,600
Birds,62.43,1.08,600
Textures,64.16,0.83,600
QuickDraw,59.73,1.1,600
Fungi,33.54,1.11,600
VGG Flower,79.94,0.84,600
Traffic signs,42.91,1.31,600


### RelationNet (`relationnet`)

In [17]:
relationnet_imagenet_df = pd.DataFrame(
    columns=['mean (%)', '95% CI', '# episodes'],
    index=datasets
)
relationnet_imagenet_df['# episodes'] = 600
relationnet_imagenet_df.loc[datasets[1:], ['mean (%)', '95% CI']] = [
    [34.69, 1.01],
    [45.35, 1.36],
    [40.73, 0.83],
    [49.51, 1.05],
    [52.97, 0.69],
    [43.30, 1.08],
    [30.55, 1.04],
    [68.76, 0.83],
    [33.67, 1.05],
    [29.15, 1.01]
]
relationnet_imagenet_df

,mean (%),95% CI,# episodes
ILSVRC (valid),NaN,NaN,600
ILSVRC (test),34.69,1.01,600
Omniglot,45.35,1.36,600
Aircraft,40.73,0.83,600
Birds,49.51,1.05,600
Textures,52.97,0.69,600
QuickDraw,43.3,1.08,600
Fungi,30.55,1.04,600
VGG Flower,68.76,0.83,600
Traffic signs,33.67,1.05,600


In [18]:
relationnet_all_df = pd.DataFrame(
    columns=['mean (%)', '95% CI', '# episodes'],
    index=datasets
)
relationnet_all_df['# episodes'] = 600
relationnet_all_df.loc[datasets[1:], ['mean (%)', '95% CI']] = [
    [30.89, 0.93],
    [86.57, 0.79],
    [69.71, 0.83],
    [54.14, 0.99],
    [56.56, 0.73],
    [61.75, 0.97],
    [32.56, 1.08],
    [76.08, 0.76],
    [37.48, 0.93],
    [27.41, 0.89]
]
relationnet_all_df

,mean (%),95% CI,# episodes
ILSVRC (valid),NaN,NaN,600
ILSVRC (test),30.89,0.93,600
Omniglot,86.57,0.79,600
Aircraft,69.71,0.83,600
Birds,54.14,0.99,600
Textures,56.56,0.73,600
QuickDraw,61.75,0.97,600
Fungi,32.56,1.08,600
VGG Flower,76.08,0.76,600
Traffic signs,37.48,0.93,600


### fo-Proto-MAML (`maml_init_with_proto`)

In [19]:
protomaml_imagenet_df = pd.DataFrame(
    columns=['mean (%)', '95% CI', '# episodes'],
    index=datasets
)
protomaml_imagenet_df['# episodes'] = 600
protomaml_imagenet_df.loc[datasets[1:], ['mean (%)', '95% CI']] = [
    [49.53, 1.05],
    [63.37, 1.33],
    [55.95, 0.99],
    [68.66, 0.96],
    [66.49, 0.83],
    [51.52, 1.00],
    [39.96, 1.14],
    [87.15, 0.69],
    [48.83, 1.09],
    [43.74, 1.12],
]
protomaml_imagenet_df

,mean (%),95% CI,# episodes
ILSVRC (valid),NaN,NaN,600
ILSVRC (test),49.53,1.05,600
Omniglot,63.37,1.33,600
Aircraft,55.95,0.99,600
Birds,68.66,0.96,600
Textures,66.49,0.83,600
QuickDraw,51.52,1,600
Fungi,39.96,1.14,600
VGG Flower,87.15,0.69,600
Traffic signs,48.83,1.09,600


In [20]:
protomaml_all_df = pd.DataFrame(
    columns=['mean (%)', '95% CI', '# episodes'],
    index=datasets
)
protomaml_all_df['# episodes'] = 600
protomaml_all_df.loc[datasets[1:], ['mean (%)', '95% CI']] = [
    [46.52, 1.05],
    [82.69, 0.97],
    [75.23, 0.76],
    [69.88, 1.02],
    [68.25, 0.81],
    [66.84, 0.94],
    [41.99, 1.17],
    [88.72, 0.67],
    [52.42, 1.08],
    [41.74, 1.13]
]
protomaml_all_df

,mean (%),95% CI,# episodes
ILSVRC (valid),NaN,NaN,600
ILSVRC (test),46.52,1.05,600
Omniglot,82.69,0.97,600
Aircraft,75.23,0.76,600
Birds,69.88,1.02,600
Textures,68.25,0.81,600
QuickDraw,66.84,0.94,600
Fungi,41.99,1.17,600
VGG Flower,88.72,0.67,600
Traffic signs,52.42,1.08,600


## Results from Requeima et al. (2019)

In [21]:
ref = ("Requeima et al. (2019)",
       "James Requeima, Jonathan Gordon, John Bronskill, Sebastian Nowozin, "
       "Richard E. Turner; "
       "[_Fast and Flexible Multi-Task Classification Using Conditional Neural "
       "Adaptive Processes_](https://arxiv.org/abs/1906.07697); "
       "NeurIPS 2019.")
references.append(ref)
# display.display(display.Markdown(ref[1]))

### CNAPs (`cnaps`)

In [22]:
cnaps_all_df = pd.DataFrame(
    columns=['mean (%)', '95% CI', '# episodes'],
    index=datasets
)
cnaps_all_df['# episodes'] = 600
cnaps_all_df.loc[datasets[1:], ['mean (%)', '95% CI']] = [
    [50.8, 1.1],
    [91.7, 0.5],
    [83.7, 0.6],
    [73.6, 0.9],
    [59.5, 0.7],
    [74.7, 0.8],
    [50.2, 1.1],
    [88.9, 0.5],
    [56.5, 1.1],
    [39.4, 1.0]
]
cnaps_all_df

,mean (%),95% CI,# episodes
ILSVRC (valid),NaN,NaN,600
ILSVRC (test),50.8,1.1,600
Omniglot,91.7,0.5,600
Aircraft,83.7,0.6,600
Birds,73.6,0.9,600
Textures,59.5,0.7,600
QuickDraw,74.7,0.8,600
Fungi,50.2,1.1,600
VGG Flower,88.9,0.5,600
Traffic signs,56.5,1.1,600


## Results from Baik et al. (2020)

In [23]:
ref = ("Baik et al. (2020)",
       "Sungyong Baik, Myungsub Choi, Janghoon Choi, Heewon Kim, Kyoung Mu Lee; "
       "[_Meta-Learning with Adaptive Hyperparameters_]"
       "(https://papers.nips.cc/paper/2020/hash/ee89223a2b625b5152132ed77abbcc79-Abstract.html); "
       "NeurIPS 2020.")
references.append(ref)
# display.display(display.Markdown(ref[1]))

### ALFA + fo-Proto-MAML (`alfa_protomaml`)

In [24]:
alfa_protomaml_imagenet_df = pd.DataFrame(
    columns=['mean (%)', '95% CI', '# episodes'],
    index=datasets
)
alfa_protomaml_imagenet_df['# episodes'] = 600
alfa_protomaml_imagenet_df.loc[datasets[1:], ['mean (%)', '95% CI']] = [
    [52.80, 1.11],
    [61.87, 1.51],
    [63.43, 1.10],
    [69.75, 1.05],
    [70.78, 0.88],
    [59.17, 1.16],
    [41.49, 1.17],
    [85.96, 0.77],
    [60.78, 1.29],
    [48.11, 1.14]
]
alfa_protomaml_imagenet_df

,mean (%),95% CI,# episodes
ILSVRC (valid),NaN,NaN,600
ILSVRC (test),52.8,1.11,600
Omniglot,61.87,1.51,600
Aircraft,63.43,1.1,600
Birds,69.75,1.05,600
Textures,70.78,0.88,600
QuickDraw,59.17,1.16,600
Fungi,41.49,1.17,600
VGG Flower,85.96,0.77,600
Traffic signs,60.78,1.29,600


### ALFA + fo-MAML (`alfa_maml`)
Not included in the global table as it performs worse than ALFA + fo-Proto-MAML overall, but provided here for reference.

In [25]:
alfa_maml_imagenet_df = pd.DataFrame(
    columns=['mean (%)', '95% CI', '# episodes'],
    index=datasets
)
alfa_maml_imagenet_df['# episodes'] = 600
alfa_maml_imagenet_df.loc[datasets[1:], ['mean (%)', '95% CI']] = [
    [51.09, 1.17],
    [67.89, 1.43],
    [66.34, 1.17],
    [67.67, 1.06],
    [65.34, 0.95],
    [60.53, 1.13],
    [37.41, 1.00],
    [84.28, 0.97],
    [60.86, 1.43],
    [40.05, 1.14]
]
alfa_maml_imagenet_df

,mean (%),95% CI,# episodes
ILSVRC (valid),NaN,NaN,600
ILSVRC (test),51.09,1.17,600
Omniglot,67.89,1.43,600
Aircraft,66.34,1.17,600
Birds,67.67,1.06,600
Textures,65.34,0.95,600
QuickDraw,60.53,1.13,600
Fungi,37.41,1,600
VGG Flower,84.28,0.97,600
Traffic signs,60.86,1.43,600


## Results from Doersch et al. (2020)
Carl Doersch, Ankush Gupta, Andrew Zisserman,
_CrossTransformers: spatially-aware few-shot transfer_,
NeurIPS 2020

In [26]:
ref = ("Doersch et al. (2020)",
       "Carl Doersch, Ankush Gupta, Andrew Zisserman; "
       "[_CrossTransformers: spatially-aware few-shot transfer_]"
       "(https://arxiv.org/abs/2007.11498); "
       "NeurIPS 2020.")
references.append(ref)
# display.display(display.Markdown(ref[1]))

### ProtoNet large (`protonet_large`)
Larger-scale prototypical networks, including:
- 224x224 input size
- ResNet-34 backbone
- SimCLR Episodes

In [27]:
protonet_large_imagenet_df = pd.DataFrame(
    columns=['mean (%)', '95% CI', '# episodes'],
    index=datasets
)
protonet_large_imagenet_df['# episodes'] = 600
protonet_large_imagenet_df.loc[datasets[1:], ['mean (%)', '95% CI']] = [
     [53.69, 1.07],
     [68.50, 1.27],
     [58.04, 0.96],
     [74.07, 0.92],
     [68.76, 0.77],
     [53.30, 1.06],
     [40.73, 1.15],
     [86.96, 0.73],
     [58.11, 1.05],
     [41.70, 1.08],
]
protonet_large_imagenet_df

,mean (%),95% CI,# episodes
ILSVRC (valid),NaN,NaN,600
ILSVRC (test),53.69,1.07,600
Omniglot,68.5,1.27,600
Aircraft,58.04,0.96,600
Birds,74.07,0.92,600
Textures,68.76,0.77,600
QuickDraw,53.3,1.06,600
Fungi,40.73,1.15,600
VGG Flower,86.96,0.73,600
Traffic signs,58.11,1.05,600


### CrossTransformers (`ctx`)

CrossTransformers network with:
- 224x224 input size
- ResNet-34 backbone
- SimCLR episodes
- 14x14 feature grid
- BOHB-inspired data augmentation

In [28]:
ctx_imagenet_df = pd.DataFrame(
    columns=['mean (%)', '95% CI', '# episodes'],
    index=datasets
)
ctx_imagenet_df['# episodes'] = 600
ctx_imagenet_df.loc[datasets[1:], ['mean (%)', '95% CI']] = [
      [62.76, 0.99],
      [82.21, 1.00],
      [79.49, 0.89],
      [80.63, 0.88],
      [75.57, 0.64],
      [72.68, 0.82],
      [51.58, 1.11],
      [95.34, 0.37],
      [82.65, 0.76],
      [59.90, 1.02],
]
ctx_imagenet_df

,mean (%),95% CI,# episodes
ILSVRC (valid),NaN,NaN,600
ILSVRC (test),62.76,0.99,600
Omniglot,82.21,1,600
Aircraft,79.49,0.89,600
Birds,80.63,0.88,600
Textures,75.57,0.64,600
QuickDraw,72.68,0.82,600
Fungi,51.58,1.11,600
VGG Flower,95.34,0.37,600
Traffic signs,82.65,0.76,600


## Results from Saikia et al. (2020)
Tonmoy Saikia, Thomas Brox, Cordelia Schmid, _Optimized Generic Feature Learning for Few-shot Classification across Domains_, arXiv 2020

In [29]:
ref = ("Saikia et al. (2020)",
       "Tonmoy Saikia, Thomas Brox, Cordelia Schmid; "
       "[_Optimized Generic Feature Learning for Few-shot Classification "
       "across Domains_]"
       "(https://arxiv.org/abs/2001.07926); "
       "arXiv 2020.")
references.append(ref)
# display.display(display.Markdown(ref[1]))

### BOHB (`bohb`)
Validated on _S1_ (ImageNet) only, nearest-centroid classifier (NC).

In [30]:
bohb_imagenet_df = pd.DataFrame(
    columns=['mean (%)', '95% CI', '# episodes'],
    index=datasets
)
bohb_imagenet_df['# episodes'] = 600
bohb_imagenet_df.loc[datasets[1:], ['mean (%)', '95% CI']] = [
    [51.92, 1.05],
    [67.57, 1.21],
    [54.12, 0.90],
    [70.69, 0.90],
    [68.34, 0.76],
    [50.33, 1.04],
    [41.38, 1.12],
    [87.34, 0.59],
    [51.80, 1.04],
    [48.03, 0.99],
]
bohb_imagenet_df

,mean (%),95% CI,# episodes
ILSVRC (valid),NaN,NaN,600
ILSVRC (test),51.92,1.05,600
Omniglot,67.57,1.21,600
Aircraft,54.12,0.9,600
Birds,70.69,0.9,600
Textures,68.34,0.76,600
QuickDraw,50.33,1.04,600
Fungi,41.38,1.12,600
VGG Flower,87.34,0.59,600
Traffic signs,51.8,1.04,600


## Results from Dvornik et al. (2020)


In [31]:
ref = ("Dvornik et al. (2020)",
       "Nikita Dvornik, Cordelia Schmid, Julien Mairal; "
       "[_Selecting Relevant Features from a Multi-domain Representation for "
       "Few-shot Classification_](https://arxiv.org/abs/2003.09338); "
       "ECCV 2020.")
references.append(ref)
# display.display(display.Markdown(ref[1]))

### SUR (`sur`)

In [32]:
sur_all_df = pd.DataFrame(
    columns=['mean (%)', '95% CI', '# episodes'],
    index=datasets
)
sur_all_df['# episodes'] = 600
sur_all_df.loc[datasets[1:], ['mean (%)', '95% CI']] = [
    [56.1, 1.1],
    [93.1, 0.5],
    [84.6, 0.7],
    [70.6, 1.0],
    [71.0, 0.8],
    [81.3, 0.6],
    [64.2, 1.1],
    [82.8, 0.8],
    [53.4, 1.0],
    [50.1, 1.0],
]
sur_all_df

,mean (%),95% CI,# episodes
ILSVRC (valid),NaN,NaN,600
ILSVRC (test),56.1,1.1,600
Omniglot,93.1,0.5,600
Aircraft,84.6,0.7,600
Birds,70.6,1,600
Textures,71,0.8,600
QuickDraw,81.3,0.6,600
Fungi,64.2,1.1,600
VGG Flower,82.8,0.8,600
Traffic signs,53.4,1,600


### SUR-pnf (`sur_pnf`)
SUR with parametric network family, also referred as "SUR-pf".

In [33]:
sur_pnf_all_df = pd.DataFrame(
    columns=['mean (%)', '95% CI', '# episodes'],
    index=datasets
)
sur_pnf_all_df['# episodes'] = 600
sur_pnf_all_df.loc[datasets[1:], ['mean (%)', '95% CI']] = [
    [56.0, 1.1],
    [90.0, 0.6],
    [79.7, 0.8],
    [75.9, 0.9],
    [72.5, 0.7],
    [76.7, 0.7],
    [49.8, 1.1],
    [90.0, 0.6],
    [52.2, 0.8],
    [50.2, 1.1],
]
sur_pnf_all_df

,mean (%),95% CI,# episodes
ILSVRC (valid),NaN,NaN,600
ILSVRC (test),56,1.1,600
Omniglot,90,0.6,600
Aircraft,79.7,0.8,600
Birds,75.9,0.9,600
Textures,72.5,0.7,600
QuickDraw,76.7,0.7,600
Fungi,49.8,1.1,600
VGG Flower,90,0.6,600
Traffic signs,52.2,0.8,600


## Results from Liu et al. (2021)

Lu Liu, William Hamilton, Guodong Long, Jing Jiang, Hugo Larochelle,
_A Universal Representation Transformer Layer for Few-Shot Image Classification_, ICLR 2021



In [34]:
ref = ("Liu et al. (2021)",
       "Lu Liu, William Hamilton, Guodong Long, Jing Jiang, Hugo Larochelle; "
       "[_Universal Representation Transformer Layer for Few-Shot Image "
       "Classification_](https://arxiv.org/abs/2006.11702); "
       "ICLR 2021.")
references.append(ref)
# display.display(display.Markdown(ref[1]))

### URT (`urt`)

In [35]:
urt_all_df = pd.DataFrame(
    columns=['mean (%)', '95% CI', '# episodes'],
    index=datasets
)
urt_all_df['# episodes'] = 600
urt_all_df.loc[datasets[1:], ['mean (%)', '95% CI']] = [
    [55.7, 1.0],
    [94.4, 0.4],
    [85.8, 0.6],
    [76.3, 0.8],
    [71.8, 0.7],
    [82.5, 0.6],
    [63.5, 1.0],
    [88.2, 0.6],
    [65.9, 1.3],
    [52.2, 1.1],
]
urt_all_df

,mean (%),95% CI,# episodes
ILSVRC (valid),NaN,NaN,600
ILSVRC (test),55.7,1,600
Omniglot,94.4,0.4,600
Aircraft,85.8,0.6,600
Birds,76.3,0.8,600
Textures,71.8,0.7,600
QuickDraw,82.5,0.6,600
Fungi,63.5,1,600
VGG Flower,88.2,0.6,600
Traffic signs,65.9,1.3,600


## Template to add a new paper

```
ref = ("Author et al. (year)",
       "First Author, Second Author, Last Author; "
       "[_Title of Paper_](https://paper.url/); "
       "Venue year.")
references.append(ref)
# display.display(display.Markdown(ref[1]))
```

### Template to add a new model

```
<model_name>_<train_source>_df = pd.DataFrame(
    columns=['mean (%)', '95% CI', '# episodes'],
    index=datasets
)
<model_name>_<train_source>_df['# episodes'] = ...
<model_name>_<train_source>_df.loc[datasets[1:], ['mean (%)', '95% CI']] = [...]
```

## Aggregate in table

In [36]:
imagenet_dfs = {
    'k-NN': baseline_imagenet_df,
    'Finetune': baselineft_imagenet_df,
    'MatchingNet': matching_imagenet_df,
    'ProtoNet': prototypical_imagenet_df,
    'fo-MAML': maml_imagenet_df,
    'RelationNet': relationnet_imagenet_df,
    'fo-Proto-MAML': protomaml_imagenet_df,
    'ALFA+fo-Proto-MAML': alfa_protomaml_imagenet_df,
    'ProtoNet (large)': protonet_large_imagenet_df,
    'CTX': ctx_imagenet_df,
    'BOHB': bohb_imagenet_df,
}

In [37]:
imagenet_df = pd.concat(
    imagenet_dfs.values(),
    axis=1,
    keys=imagenet_dfs.keys())
imagenet_df

k-NN                    ...     BOHB                  
               mean (%) 95% CI # episodes  ... mean (%) 95% CI # episodes
ILSVRC (valid)      NaN    NaN        600  ...      NaN    NaN        600
ILSVRC (test)     41.03   1.01        600  ...    51.92   1.05        600
Omniglot          37.07   1.15        600  ...    67.57   1.21        600
Aircraft          46.81   0.89        600  ...    54.12    0.9        600
Birds             50.13      1        600  ...    70.69    0.9        600
Textures          66.36   0.75        600  ...    68.34   0.76        600
QuickDraw         32.06   1.08        600  ...    50.33   1.04        600
Fungi             36.16   1.02        600  ...    41.38   1.12        600
VGG Flower         83.1   0.68        600  ...    87.34   0.59        600
Traffic signs     44.59   1.19        600  ...     51.8   1.04        600
MSCOCO            30.38   0.99        600  ...    48.03   0.99        600

[11 rows x 33 columns]

In [38]:
all_dfs = {
    'k-NN': baseline_all_df,
    'Finetune': baselineft_all_df,
    'MatchingNet': matching_all_df,
    'ProtoNet': prototypical_all_df,
    'fo-MAML': maml_all_df,
    'RelationNet': relationnet_all_df,
    'fo-Proto-MAML': protomaml_all_df,
    'CNAPs': cnaps_all_df,
    'SUR': sur_all_df,
    'SUR-pnf': sur_pnf_all_df,
    'URT': urt_all_df,
}
all_df = pd.concat(
    all_dfs.values(),
    axis=1,
    keys=all_dfs.keys())
all_df

k-NN                    ...      URT                  
               mean (%) 95% CI # episodes  ... mean (%) 95% CI # episodes
ILSVRC (valid)      NaN    NaN        600  ...      NaN    NaN        600
ILSVRC (test)     38.55   0.94        600  ...     55.7      1        600
Omniglot           74.6   1.08        600  ...     94.4    0.4        600
Aircraft          64.98   0.82        600  ...     85.8    0.6        600
Birds             66.35   0.92        600  ...     76.3    0.8        600
Textures          63.58   0.79        600  ...     71.8    0.7        600
QuickDraw         44.88   1.05        600  ...     82.5    0.6        600
Fungi             37.12   1.06        600  ...     63.5      1        600
VGG Flower        83.47   0.61        600  ...     88.2    0.6        600
Traffic signs     40.11    1.1        600  ...     65.9    1.3        600
MSCOCO            29.55   0.96        600  ...     52.2    1.1        600

[11 rows x 33 columns]

In [39]:
models_df = pd.DataFrame.from_dict(
    orient='index',
    columns=["ref"],
    data={
        'k-NN': 'Triantafillou et al. (2020)',
        'Finetune': 'Triantafillou et al. (2020)',
        'MatchingNet': 'Triantafillou et al. (2020)',
        'ProtoNet': 'Triantafillou et al. (2020)',
        'fo-MAML': 'Triantafillou et al. (2020)',
        'RelationNet': 'Triantafillou et al. (2020)',
        'fo-Proto-MAML': 'Triantafillou et al. (2020)',
        'CNAPs': 'Requeima et al. (2019)',
        'ALFA+fo-Proto-MAML': 'Baik et al. (2020)',
        'ProtoNet (large)': 'Doersch et al. (2020)',
        'CTX': 'Doersch et al. (2020)',
        'BOHB': 'Saikia et al. (2020)',
        'SUR': 'Dvornik et al. (2020)',
        'SUR-pnf': 'Dvornik et al. (2020)',
        'URT': 'Liu et al. (2021)',
        })
models_df

,ref
k-NN,Triantafillou et al. (2020)
Finetune,Triantafillou et al. (2020)
MatchingNet,Triantafillou et al. (2020)
ProtoNet,Triantafillou et al. (2020)
fo-MAML,Triantafillou et al. (2020)
RelationNet,Triantafillou et al. (2020)
fo-Proto-MAML,Triantafillou et al. (2020)
CNAPs,Requeima et al. (2019)
ALFA+fo-Proto-MAML,Baik et al. (2020)
ProtoNet (large),Doersch et al. (2020)


### Add stddev

In [40]:
def add_stddev(df):
  # Extract original order of labels
  datasets = df.index
  models = df.columns.levels[0]
  # Have only one result (mean, CI, ...) per row
  stacked_df = df.stack(0)
  # Add 'stddev' as column
  stacked_df['stddev'] = stacked_df['95% CI'] * np.sqrt(stacked_df['# episodes']) / 1.96
  # Reshape and put back in original order
  new_df = stacked_df.unstack().swaplevel(0, 1, axis=1)
  new_df = new_df.loc[datasets][models]
  return new_df

In [41]:
imagenet_df = add_stddev(imagenet_df)
imagenet_df

k-NN                  ...   BOHB                  
               # episodes 95% CI mean (%)  ... 95% CI mean (%)   stddev
ILSVRC (valid)        600    NaN      NaN  ...    NaN      NaN      NaN
ILSVRC (test)         600   1.01    41.03  ...   1.05    51.92  13.1223
Omniglot              600   1.15    37.07  ...   1.21    67.57  15.1218
Aircraft              600   0.89    46.81  ...    0.9    54.12  11.2477
Birds                 600      1    50.13  ...    0.9    70.69  11.2477
Textures              600   0.75    66.36  ...   0.76    68.34  9.49802
QuickDraw             600   1.08    32.06  ...   1.04    50.33  12.9973
Fungi                 600   1.02    36.16  ...   1.12    41.38  13.9971
VGG Flower            600   0.68     83.1  ...   0.59    87.34  7.37346
Traffic signs         600   1.19    44.59  ...   1.04     51.8  12.9973
MSCOCO                600   0.99    30.38  ...   0.99    48.03  12.3724

[11 rows x 44 columns]

In [42]:
all_df = add_stddev(all_df)
all_df

k-NN                  ...    URT                  
               # episodes 95% CI mean (%)  ... 95% CI mean (%)   stddev
ILSVRC (valid)        600    NaN      NaN  ...    NaN      NaN      NaN
ILSVRC (test)         600   0.94    38.55  ...      1     55.7  12.4974
Omniglot              600   1.08     74.6  ...    0.4     94.4  4.99896
Aircraft              600   0.82    64.98  ...    0.6     85.8  7.49844
Birds                 600   0.92    66.35  ...    0.8     76.3  9.99792
Textures              600   0.79    63.58  ...    0.7     71.8  8.74818
QuickDraw             600   1.05    44.88  ...    0.6     82.5  7.49844
Fungi                 600   1.06    37.12  ...      1     63.5  12.4974
VGG Flower            600   0.61    83.47  ...    0.6     88.2  7.49844
Traffic signs         600    1.1    40.11  ...    1.3     65.9  16.2466
MSCOCO                600   0.96    29.55  ...    1.1     52.2  13.7471

[11 rows x 44 columns]

### Add rankings

In [43]:
def is_difference_significant(best_stats, candidate_stats):
  # compute a 95% confidence for the difference of means.
  ci = 1.96 * np.sqrt((best_stats['stddev'] ** 2) / best_stats['# episodes'] +
                      (candidate_stats['stddev'] ** 2) / candidate_stats['# episodes'])
  diff_of_means = best_stats['mean (%)'] - candidate_stats['mean (%)']
  return np.abs(diff_of_means) > ci

In [44]:
def compute_ranks(dataset_series):
  dataset_df = dataset_series.unstack()
  n_models = len(dataset_df.index)
  remaining_models = list(dataset_df.index)
  next_available_rank = 1
  ranks = {}
  # Iteratively pick the best models, then all the ones statistically equivalent
  while remaining_models:
    accuracies = dataset_df.loc[remaining_models]['mean (%)'].astype('d')
    best_model = accuracies.idxmax(axis=1)
    best_stats = dataset_df.loc[best_model]
    tied_models = [best_model]
    potential_tied_models = [model for model in remaining_models
                             if model != best_model]
    for candidate in potential_tied_models:
      candidate_stats = dataset_df.loc[candidate]
      if not is_difference_significant(best_stats, candidate_stats):
        tied_models.append(candidate)

    n_ties = len(tied_models)
    # All tied models share the same rank, which is the average of the next
    # `n_ties` available ranks (the ranks they would have without the ties), or
    # next_available_rank + (1 + ... + (n_ties - 1)) / n_ties, which gives:
    shared_rank = next_available_rank + (n_ties - 1) / 2
    next_available_rank += n_ties
    for model in tied_models:
      ranks[model] = shared_rank

    # Remove picked models for next iteration
    remaining_models = [model for model in remaining_models
                        if model not in tied_models]
  return pd.Series(ranks, name='rank')

In [45]:
def add_ranks(df):
  # Get ranks as a data frame (ignore "ILSVRC (valid)")
  ranks = df[1:].apply(compute_ranks, axis=1)
  # Set the columns as (model, 'rank') Multi-index
  ranks = pd.concat([ranks], axis=1, keys=['rank']).swaplevel(0, 1, axis=1)
  # Concatenate with the original dataframe and defrag columns
  new_df = pd.concat([df, ranks], axis=1)[df.columns.levels[0]]
  return new_df

In [46]:
imagenet_df = add_ranks(imagenet_df)
imagenet_df

ALFA+fo-Proto-MAML                  ...     k-NN               
                       # episodes 95% CI mean (%)  ... mean (%)   stddev  rank
ILSVRC (valid)                600    NaN      NaN  ...      NaN      NaN   NaN
ILSVRC (test)                 600   1.11     52.8  ...    41.03  12.6224  10.0
Omniglot                      600   1.51    61.87  ...    37.07   14.372  11.0
Aircraft                      600    1.1    63.43  ...    46.81  11.1227  10.0
Birds                         600   1.05    69.75  ...    50.13  12.4974  10.5
Textures                      600   0.88    70.78  ...    66.36  9.37305   8.0
QuickDraw                     600   1.16    59.17  ...    32.06  13.4972  11.0
Fungi                         600   1.17    41.49  ...    36.16  12.7473   8.0
VGG Flower                    600   0.77    85.96  ...     83.1  8.49823   8.0
Traffic signs                 600   1.29    60.78  ...    44.59  14.8719  10.0
MSCOCO                        600   1.14    48.11  ...    30.38  12.3724  10.5

[11 rows x 55 columns]

In [47]:
all_df = add_ranks(all_df)
all_df

CNAPs                  ...     k-NN               
               # episodes 95% CI mean (%)  ... mean (%)   stddev  rank
ILSVRC (valid)        600    NaN      NaN  ...      NaN      NaN   NaN
ILSVRC (test)         600    1.1     50.8  ...    38.55  11.7476   8.5
Omniglot              600    0.5     91.7  ...     74.6  13.4972  10.0
Aircraft              600    0.6     83.7  ...    64.98  10.2479  11.0
Birds                 600    0.9     73.6  ...    66.35  11.4976   6.5
Textures              600    0.7     59.5  ...    63.58  9.87294   8.0
QuickDraw             600    0.8     74.7  ...    44.88  13.1223  11.0
Fungi                 600    1.1     50.2  ...    37.12  13.2472   7.5
VGG Flower            600    0.5     88.9  ...    83.47  7.62341   7.5
Traffic signs         600    1.1     56.5  ...    40.11  13.7471  10.0
MSCOCO                600      1     39.4  ...    29.55  11.9975   9.0

[11 rows x 55 columns]

In [48]:
imagenet_df.xs('rank', axis=1, level=1).mean()

ALFA+fo-Proto-MAML     3.25
BOHB                   4.15
CTX                    1.00
Finetune               6.15
MatchingNet            8.65
ProtoNet               6.45
ProtoNet (large)       3.45
RelationNet           10.55
fo-MAML                7.45
fo-Proto-MAML          5.20
k-NN                   9.70
dtype: float64

In [49]:
all_df.xs('rank', axis=1, level=1).mean()

CNAPs            4.20
Finetune         7.05
MatchingNet      8.60
ProtoNet         6.60
RelationNet      9.80
SUR              3.35
SUR-pnf          2.90
URT              1.50
fo-MAML          8.15
fo-Proto-MAML    4.95
k-NN             8.90
dtype: float64

### Display in HTML
This section uses the DataFrame's "styler" object, which renders nicely within the notebook.

Unfortunately, the HTML it outputs is not compatible with GitHub's markdown (as it relies on the `<style>` tag).

In [50]:
def str_summary(series):
  # Summarize each (episode, model) by a single cell
  # Non-breaking space to keep things on the same line
  nbsp = '\u00A0'
  string = '%(acc)s±%(ci)s%(nbsp)s(%(rank)g)' % {
      'acc': series['mean (%)'],
      'ci': series['95% CI'],
      'rank': series['rank'],
      'nbsp': nbsp
  }
  return string

In [51]:
def display_table(df, models=None):
  accuracies_df = df.stack(0).apply(str_summary, axis=1).unstack(0)[df.index[1:]]
  rank_df = df.xs('rank', axis=1, level=1).loc[df.index[1:]]
  avg_rank_df = pd.DataFrame(rank_df.mean(), columns=['Avg rank'])
  display_df = pd.concat([avg_rank_df, accuracies_df], axis=1)
  if models:
    # Try and force a particular order of models
    display_df = display_df.loc[models]
 
  # Bold cells corresponding to the best rank
  best_acc_mask = rank_df.T == rank_df.min(axis=1)
  best_avg_mask = avg_rank_df == avg_rank_df.min()
  best_mask = pd.concat([best_avg_mask, best_acc_mask], axis=1)
  if models:
    best_mask = best_mask.loc[models]
  bold_mask = best_mask.applymap(lambda v: 'font-weight: bold' if v else '')

  display_style = display_df.style.apply(lambda f: bold_mask, axis=None)
  display_style = display_style.format({'Avg rank': '{:g}'})
  return display_style

In [52]:
imagenet_display = display_table(imagenet_df, models=imagenet_dfs.keys())
imagenet_display

,Avg rank,ILSVRC (test),Omniglot,Aircraft,Birds,Textures,QuickDraw,Fungi,VGG Flower,Traffic signs,MSCOCO
k-NN,9.7,41.03±1.01 (10),37.07±1.15 (11),46.81±0.89 (10),50.13±1.0 (10.5),66.36±0.75 (8),32.06±1.08 (11),36.16±1.02 (8),83.1±0.68 (8),44.59±1.19 (10),30.38±0.99 (10.5)
Finetune,6.15,45.78±1.1 (8),60.85±1.58 (6.5),68.69±1.26 (2),57.31±1.26 (9),69.05±0.9 (4.5),42.6±1.17 (8.5),38.2±1.02 (7),85.51±0.68 (6),66.79±1.31 (2),34.86±0.97 (8)
MatchingNet,8.65,45.0±1.1 (8),52.27±1.28 (9),48.97±0.93 (9),62.21±0.95 (7.5),64.15±0.85 (10),42.87±1.09 (8.5),33.97±1.0 (9),80.13±0.71 (10),47.8±1.14 (7.5),34.99±1.0 (8)
ProtoNet,6.45,50.5±1.08 (4.5),59.98±1.35 (6.5),53.1±1.0 (7.5),68.79±1.01 (5.5),66.56±0.83 (8),48.96±1.08 (6),39.71±1.11 (6),85.27±0.77 (6),47.12±1.1 (9),41.0±1.1 (5.5)
fo-MAML,7.45,45.51±1.11 (8),55.55±1.54 (8),56.24±1.11 (5.5),63.61±1.06 (7.5),68.04±0.81 (4.5),43.96±1.29 (8.5),32.1±1.1 (10),81.74±0.83 (9),50.93±1.51 (5.5),35.3±1.23 (8)
RelationNet,10.55,34.69±1.01 (11),45.35±1.36 (10),40.73±0.83 (11),49.51±1.05 (10.5),52.97±0.69 (11),43.3±1.08 (8.5),30.55±1.04 (11),68.76±0.83 (11),33.67±1.05 (11),29.15±1.01 (10.5)
fo-Proto-MAML,5.2,49.53±1.05 (6),63.37±1.33 (4.5),55.95±0.99 (5.5),68.66±0.96 (5.5),66.49±0.83 (8),51.52±1.0 (4.5),39.96±1.14 (3.5),87.15±0.69 (3),48.83±1.09 (7.5),43.74±1.12 (4)
ALFA+fo-Proto-MAML,3.25,52.8±1.11 (2.5),61.87±1.51 (4.5),63.43±1.1 (3),69.75±1.05 (3.5),70.78±0.88 (2),59.17±1.16 (2),41.49±1.17 (3.5),85.96±0.77 (6),60.78±1.29 (3),48.11±1.14 (2.5)
ProtoNet (large),3.45,53.69±1.07 (2.5),68.5±1.27 (2.5),58.04±0.96 (4),74.07±0.92 (2),68.76±0.77 (4.5),53.3±1.06 (3),40.73±1.15 (3.5),86.96±0.73 (3),58.11±1.05 (4),41.7±1.08 (5.5)
CTX,1,62.76±0.99 (1),82.21±1.0 (1),79.49±0.89 (1),80.63±0.88 (1),75.57±0.64 (1),72.68±0.82 (1),51.58±1.11 (1),95.34±0.37 (1),82.65±0.76 (1),59.9±1.02 (1)


In [53]:
# print(imagenet_display.render())

In [54]:
all_display = display_table(all_df, models=all_dfs.keys())
all_display

,Avg rank,ILSVRC (test),Omniglot,Aircraft,Birds,Textures,QuickDraw,Fungi,VGG Flower,Traffic signs,MSCOCO
k-NN,8.9,38.55±0.94 (8.5),74.6±1.08 (10),64.98±0.82 (11),66.35±0.92 (6.5),63.58±0.79 (8),44.88±1.05 (11),37.12±1.06 (7.5),83.47±0.61 (7.5),40.11±1.1 (10),29.55±0.96 (9)
Finetune,7.05,43.08±1.08 (6.5),71.11±1.37 (11),72.03±1.07 (7.5),59.82±1.15 (9),69.14±0.85 (4.5),47.05±1.16 (10),38.16±1.04 (7.5),85.28±0.69 (6),66.74±1.23 (1.5),35.17±1.08 (7)
MatchingNet,8.6,36.08±1.0 (10),78.25±1.01 (8.5),69.17±0.96 (9.5),56.4±1.0 (10),61.8±0.74 (9),60.81±1.03 (7.5),33.7±1.04 (10),81.9±0.72 (9),55.57±1.08 (3.5),28.79±0.96 (9)
ProtoNet,6.6,44.5±1.05 (6.5),79.56±1.12 (8.5),71.14±0.86 (7.5),67.01±1.02 (6.5),65.18±0.84 (6.5),64.88±0.89 (6),40.26±1.13 (6),86.85±0.71 (5),46.48±1.0 (8),39.87±1.06 (5.5)
fo-MAML,8.15,37.83±1.01 (8.5),83.92±0.95 (6.5),76.41±0.69 (5),62.43±1.08 (8),64.16±0.83 (6.5),59.73±1.1 (9),33.54±1.11 (10),79.94±0.84 (10),42.91±1.31 (9),29.37±1.08 (9)
RelationNet,9.8,30.89±0.93 (11),86.57±0.79 (5),69.71±0.83 (9.5),54.14±0.99 (11),56.56±0.73 (11),61.75±0.97 (7.5),32.56±1.08 (10),76.08±0.76 (11),37.48±0.93 (11),27.41±0.89 (11)
fo-Proto-MAML,4.95,46.52±1.05 (5),82.69±0.97 (6.5),75.23±0.76 (6),69.88±1.02 (4.5),68.25±0.81 (4.5),66.84±0.94 (5),41.99±1.17 (5),88.72±0.67 (3),52.42±1.08 (6),41.74±1.13 (4)
CNAPs,4.2,50.8±1.1 (4),91.7±0.5 (3),83.7±0.6 (2.5),73.6±0.9 (3),59.5±0.7 (10),74.7±0.8 (4),50.2±1.1 (3.5),88.9±0.5 (3),56.5±1.1 (3.5),39.4±1.0 (5.5)
SUR,3.35,56.1±1.1 (2),93.1±0.5 (2),84.6±0.7 (2.5),70.6±1.0 (4.5),71.0±0.8 (3),81.3±0.6 (2),64.2±1.1 (1.5),82.8±0.8 (7.5),53.4±1.0 (6),50.1±1.0 (2.5)
SUR-pnf,2.9,56.0±1.1 (2),90.0±0.6 (4),79.7±0.8 (4),75.9±0.9 (1.5),72.5±0.7 (1.5),76.7±0.7 (3),49.8±1.1 (3.5),90.0±0.6 (1),52.2±0.8 (6),50.2±1.1 (2.5)


In [55]:
# print(all_display.render())

### Display in MarkDown
At least, in GitHub-flavored MarkDown.

In [56]:
def md_render(series):
  # Summarize each (episode, model) by a single cell containing MarkDown
  nbsp = '&nbsp;'
  md_string = '%(bold)s%(acc)5.2f%(bold)s±%(ci)4.2f%(nbsp)s(%(rank)g)' % {
      'acc': series['mean (%)'],
      'ci': series['95% CI'],
      'rank': series['rank'],
      'bold': '**' if series['best_rank'] else '',
      'nbsp': nbsp
  }
  return md_string

In [57]:
def md_table(df, models=None):
  # Whether a model has the best rank on a given dataset
  rank_df = df.xs('rank', axis=1, level=1).loc[df.index[1:]]
  best_rank = pd.concat([rank_df.T == rank_df.min(axis=1)], axis=1,
                        keys=['best_rank']).swaplevel(0, 1, axis=1)
  accuracies_df = df[1:].T.unstack(1)
  accuracies_df = pd.concat([accuracies_df, best_rank], axis=1)
  accuracies_md = accuracies_df.stack(0).apply(md_render, axis=1).unstack(1)

  # Average rank (and whether it's the best)
  avg_rank_df = pd.DataFrame(rank_df.mean(), columns=['Avg rank'])
  best_avg_rank = (avg_rank_df == avg_rank_df.min()).rename(
      columns={'Avg rank': 'best_rank'})
  avg_rank_md = pd.concat([avg_rank_df, best_avg_rank], axis=1).apply(
      lambda s: '%(bold)s%(avg_rank)g%(bold)s' % {
          'avg_rank': s['Avg rank'],
          'bold': '**' if s['best_rank'] else ''
      },
      axis=1).rename('Avg rank')

  # Display method name with a pointer to the reference, defined later.
  ref_to_link = {ref[0]: "[[%i]]" % i for i, ref in enumerate(references, 1)}
  method_md = models_df.apply(lambda r: ref_to_link[r['ref']], axis='columns')

  display_md = pd.concat([avg_rank_md, accuracies_md[df.index[1:]]], axis=1)
  if models:
    # Try and force a particular order of models
    display_md = display_md.loc[list(models)]

  # Pad all cells so they align well, 27 chars should be enough
  header_str = '|'.join(['%-27s' % c
                         for c in ['Method'] + list(display_md.columns)])
  sep_str = '|'.join(['-' * 27 for _ in [''] + list(display_md.columns)])  
  rows = [
      '|'.join(['%-27s' % c for c in ([' '.join((i, method_md.loc[i]))] +
                                      list(display_md.loc[i]))])
      for i in display_md.index
  ]
  return '\n'.join([header_str, sep_str] + rows)

In [58]:
print(md_table(imagenet_df, models=imagenet_dfs.keys()))

Method                     |Avg rank                   |ILSVRC (test)              |Omniglot                   |Aircraft                   |Birds                      |Textures                   |QuickDraw                  |Fungi                      |VGG Flower                 |Traffic signs              |MSCOCO                     
---------------------------|---------------------------|---------------------------|---------------------------|---------------------------|---------------------------|---------------------------|---------------------------|---------------------------|---------------------------|---------------------------|---------------------------
k-NN [[1]]                 |9.7                        |41.03±1.01&nbsp;(10)       |37.07±1.15&nbsp;(11)       |46.81±0.89&nbsp;(10)       |50.13±1.00&nbsp;(10.5)     |66.36±0.75&nbsp;(8)        |32.06±1.08&nbsp;(11)       |36.16±1.02&nbsp;(8)        |83.10±0.68&nbsp;(8)        |44.59±1.19&nbsp;(10)       |30.38±0.99&nbsp;(10.

In [59]:
print(md_table(all_df, models=all_dfs.keys()))

Method                     |Avg rank                   |ILSVRC (test)              |Omniglot                   |Aircraft                   |Birds                      |Textures                   |QuickDraw                  |Fungi                      |VGG Flower                 |Traffic signs              |MSCOCO                     
---------------------------|---------------------------|---------------------------|---------------------------|---------------------------|---------------------------|---------------------------|---------------------------|---------------------------|---------------------------|---------------------------|---------------------------
k-NN [[1]]                 |8.9                        |38.55±0.94&nbsp;(8.5)      |74.60±1.08&nbsp;(10)       |64.98±0.82&nbsp;(11)       |66.35±0.92&nbsp;(6.5)      |63.58±0.79&nbsp;(8)        |44.88±1.05&nbsp;(11)       |37.12±1.06&nbsp;(7.5)      |83.47±0.61&nbsp;(7.5)      |40.11±1.10&nbsp;(10)       |29.55±0.96&nbsp;(9) 

## Export to MarkDown

### Reference list

In [60]:
def sanitize_anchor(string):
  # Try to mimic the MarkDown function that transforms a section title into an
  # html link anchor, that is:
  # - put it in lower case
  # - remove everything that is not a text character ("\w", which includes "_"),
  #   a space ("\s") or dash ("-")
  # - replace spaces and "_" by "-" (and deduplicate)
  anchor = string.lower()
  anchor = re.sub('[^\w\s-]', '', anchor)
  anchor = re.sub('[\s_-]+', '-', anchor)
  return anchor

In [61]:
def ref_list():
  # Define links from [i] to the reference section
  links = []
  for i, ref in enumerate(references, 1):
    links.append('[%(i)i]: #%(i)i-%(r)s' % dict(
        i=i,
        r=sanitize_anchor(ref[0])))

  references_md = []
  # Content of the reference section
  for i, ref in enumerate(references, 1):
    references_md.append(textwrap.dedent(r'''
      ###### \[%(i)i\] %(shortref)s

      %(fullref)s
    ''') % dict(i=i, shortref=ref[0], fullref=ref[1]))

  return '\n'.join(links + references_md)

In [62]:
print(ref_list())

[1]: #1-triantafillou-et-al-2020
[2]: #2-requeima-et-al-2019
[3]: #3-baik-et-al-2020
[4]: #4-doersch-et-al-2020
[5]: #5-saikia-et-al-2020
[6]: #6-dvornik-et-al-2020
[7]: #7-liu-et-al-2021

###### \[1\] Triantafillou et al. (2020)

Eleni Triantafillou, Tyler Zhu, Vincent Dumoulin, Pascal Lamblin, Utku Evci, Kelvin Xu, Ross Goroshin, Carles Gelada, Kevin Swersky, Pierre-Antoine Manzagol, Hugo Larochelle; [_Meta-Dataset: A Dataset of Datasets for Learning to Learn from Few Examples_](https://arxiv.org/abs/1903.03096); ICLR 2020.


###### \[2\] Requeima et al. (2019)

James Requeima, Jonathan Gordon, John Bronskill, Sebastian Nowozin, Richard E. Turner; [_Fast and Flexible Multi-Task Classification Using Conditional Neural Adaptive Processes_](https://arxiv.org/abs/1906.07697); NeurIPS 2019.


###### \[3\] Baik et al. (2020)

Sungyong Baik, Myungsub Choi, Janghoon Choi, Heewon Kim, Kyoung Mu Lee; [_Meta-Learning with Adaptive Hyperparameters_](https://papers.nips.cc/paper/2020/hash/ee89223

### Full section

In [63]:
def export_md():
  begin_line = '<!-- Beginning of content generated by `Leaderboard.ipynb` -->'
  end_line = '<!-- End of content generated by `Leaderboard.ipynb` -->'

  parts = [
      begin_line,
      '## Training on ImageNet only',
      md_table(imagenet_df, models=imagenet_dfs.keys()),
      '## Training on all datasets',
      md_table(all_df, models=all_dfs.keys()),
      '## References',
      ref_list(),
      end_line
  ]
  return '\n\n'.join(parts)

In [64]:
print(export_md())

<!-- Beginning of content generated by `Leaderboard.ipynb` -->

## Training on ImageNet only

Method                     |Avg rank                   |ILSVRC (test)              |Omniglot                   |Aircraft                   |Birds                      |Textures                   |QuickDraw                  |Fungi                      |VGG Flower                 |Traffic signs              |MSCOCO                     
---------------------------|---------------------------|---------------------------|---------------------------|---------------------------|---------------------------|---------------------------|---------------------------|---------------------------|---------------------------|---------------------------|---------------------------
k-NN [[1]]                 |9.7                        |41.03±1.01&nbsp;(10)       |37.07±1.15&nbsp;(11)       |46.81±0.89&nbsp;(10)       |50.13±1.00&nbsp;(10.5)     |66.36±0.75&nbsp;(8)        |32.06±1.08&nbsp;(11)       |36.16±1.02